In [ ]:
import pandas as pd
import numpy as np
import pathlib  
import featuretools as ft
import re
import multiprocessing

np.random.seed(42)
ft.__version__

# Retrieve and clean up data

In [ ]:
def extract_charge_type(k):
    try:
        return re.findall(r"Charge Filed Charge #?1: (.+)", k)[0]
    except IndexError as e:
        return np.nan

def extract_judge(k):
    """ TODO: this way will extract judge like COURTROOM C-10, which is just where they are arraigned.
    """
    try:
        return re.findall(r"^Event Scheduled Event.+Judge\:\s(.+)\sLocation", k)[0]
    except IndexError as e:
        return np.nan

def get_docket(path):
    docket = pd.read_pickle('%s/docket.dat' % path, compression='gzip')
    docket = docket.drop(columns=['Image_Avail','Docket_Index'])
    docket["Docket_Text"] = docket["Docket_Text"].fillna('')
    docket['Date'] = pd.to_datetime(docket['Date'], errors='coerce').copy()

    docket['fta'] = docket['Docket_Text'].str.contains('Defendant Failed to Appear')
    docket['fta'] = docket['fta'] * 1
    docket["fta"] = docket["fta"].astype("int")

    docket["charge_type"] = docket["Docket_Text"].apply(extract_charge_type)

    docket["judge"] = docket["Docket_Text"].apply(extract_judge)
    judge_in_first_event = docket.drop_duplicates(subset=["Case_ID"], keep="first")["judge"]
    index_of_missing_judge_first_event = judge_in_first_event[judge_in_first_event.isnull()]
    docket.loc[index_of_missing_judge_first_event.index, "judge"] = ""
    docket["judge"].fillna(method='ffill', inplace=True)
    
    docket.drop(columns="Docket_Text", inplace=True)
    docket.reset_index(inplace=True, drop=True)
    return docket
    
def get_summary(docket ,path):
    summary = pd.read_pickle('%s/summary.dat' % path, compression='gzip')
    summary = summary[["Case_ID", "Case_Type", "File_Date"]].reset_index(drop=True).copy()
    summary['File_Date'] = pd.to_datetime(summary['File_Date']).copy()

    cases = summary.copy()
    cases.index = cases.Case_ID.copy()
    label = docket.iloc[np.where(docket["fta"].values == 1)].drop_duplicates(subset="Case_ID", keep="first")[["Case_ID", "fta"]].reset_index(drop=True)
    cases = cases.join(label.set_index('Case_ID')).copy()
    summary = cases.drop_duplicates(subset='Case_ID',keep='first')
    summary['fta'] = summary['fta'].fillna(0).astype(int)
    return summary.reset_index(drop=True)


def get_receipts(path):
    receipts = pd.read_pickle('%s/receipts.dat' % path, compression='gzip')
    receipts = receipts.drop(columns=['Receipts_Index', 'Received_From']).copy()
    receipts['Payment_Amount'] = pd.to_numeric(receipts['Payment_Amount'].str.replace('$', ''), errors='coerce')
    receipts['Receipt_Date'] = pd.to_datetime(receipts['Receipt_Date'], errors='coerce')
    return receipts

def get_party(path):
    party = pd.read_pickle('%s/party.dat' % path, compression='gzip')
    party = party.drop(columns=['party_type', 'party_akas', 'Party_Index']).copy()
    return party


def dfs_from_partition(path):
    docket = get_docket(path)
    summary = get_summary(docket,path)
    docket = docket.copy()
    receipts = get_receipts(path)
    party = get_party(path)
    return docket, summary, receipts, party


In [ ]:
path = pathlib.Path.cwd() / 'partitions' / ('p%s' % str(0).zfill(2))
docket, summary, receipts, party = dfs_from_partition(path)

In [ ]:
for df in [docket, summary, receipts, party]:
    print(df.columns)
    print(df.head())

# Create entity set from data

In [ ]:
def entityset_from_partition(partition_number):
    path = pathlib.Path.cwd() / 'partitions' / ('p%s' % str(partition_number).zfill(2))

    """Create an EntitySet from a partition of data"""
    docket, summary, receipts, party = dfs_from_partition(path)
    case = summary.merge(party[["party_label","Case_ID"]], on="Case_ID").drop_duplicates(subset='Case_ID',  keep='first').copy()


    es = ft.EntitySet(id="Cases")

    es = es.entity_from_dataframe(
        entity_id = "docket", 
        dataframe = docket,
        index = 'docket_id',
        time_index="Date",
        variable_types = {
            'Date': ft.variable_types.TimeIndex,
            'Case_ID': ft.variable_types.Categorical,
            "charge_type": ft.variable_types.Categorical,
            "judge": ft.variable_types.Categorical,
        }, 
    )

    es = es.entity_from_dataframe(
        entity_id = "case", 
        dataframe = case,
        variable_types = {
            "File_Date": ft.variable_types.TimeIndex,
            "party_label": ft.variable_types.FullName,
            'fta': ft.variable_types.Numeric, # fta is used here for historical stats. we will drop some of them ro prevent data leakage.
        }, 
        index = 'Case_ID',
        time_index="File_Date",
    )

    es = es.add_relationship(
        ft.Relationship(
            es["case"]["Case_ID"],
            es["docket"]["Case_ID"]
        )
    )

    es.normalize_entity("case","party","party_label")

    
    es = es.entity_from_dataframe(
        entity_id = "receipts", 
        dataframe = receipts,
        index="receipt_index", 
        variable_types={
             'Receipt_Date': ft.variable_types.TimeIndex, 
             'Payment_Amount': ft.variable_types.Numeric, 
         },
    )

    es = es.add_relationship(ft.Relationship(es["case"]["Case_ID"],es["receipts"]["Case_ID"]))

    es = es.entity_from_dataframe(
        entity_id = "attorney", 
        dataframe = party,
        make_index = True, 
        index = 'attorney_index',
        variable_types = {
            'party_label': ft.variable_types.FullName,
            'party_attorney': ft.variable_types.FullName,
        }, 
    )

    es = es.add_relationship(
        ft.Relationship(
            es["party"]["party_label"],
            es["attorney"]["party_label"],

        )
    )

#     es.normalize_entity(base_entity_id='docket', new_entity_id='judges', index='judge')

    es_dict = {'es':es, 'partition_number':partition_number}
    print(es)
    es.plot()
    return es_dict


In [ ]:
def feature_matrix_from_entityset(es_dict):
    # Extract the entityset
    es = es_dict['es']
    cutoff_times = es['case'].df[['Case_ID', 'File_Date', 'fta']]
    # Calculate the feature matrix and save
    fm, features = ft.dfs(
        entityset=es, 
        n_jobs = multiprocessing.cpu_count(), 
        max_depth = 3,
        approximate='12h',
        cutoff_time = cutoff_times,
        verbose = True,
        target_entity='case',
        agg_primitives=[
                    "count",
                    "sum",
                    'percent_true',
                    "time_since_last",
                    "time_since_first",
                    "num_unique",
                ],
        trans_primitives=[
                    "cum_count", 
                    "year", 
                ],
        )

    return fm, features


In [ ]:
es_dict = entityset_from_partition(1)
es_dict['es'].plot()


In [ ]:
partition_numbers = [i for i in range(6,61)]
for partition_number in partition_numbers:
    print('partition', partition_number, 'of', 'partition_numbers')
    es_dict = entityset_from_partition(partition_number)
    fm, features = feature_matrix_from_entityset(es_dict)
    fm.to_pickle(pathlib.Path.cwd() / 'partitions' / 'fms' / ('fm%s.dat' % str(partition_number).zfill(2)), compression='gzip')


In [ ]:
print("Done!")